In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install -U evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate
import torch

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/ML_DL_datasets/VSFC/human_annotator/train.csv')[2000:]
df_train.head()

,sentence,label,idx
2000,dạy quá thời gian quy định .,2,2000
2001,"giảng viên dạy dễ hiểu , vui tính .",2,2001
2002,"thầy dạy khá nâng cao , một số tụi em học khôn...",0,2002
2003,sinh viên cần nhiều hơn những bài như vậy .,0,2003
2004,thầy tận dụng tốt thời gian và truyền đạt ý tư...,2,2004


In [ ]:
# # Encode labels (positive=0, negative=1, neutral=2)
# le = LabelEncoder()
# df_train['label_encoded'] = le.fit_transform(df_train['label'])
# df_train.head()

In [ ]:
# Split data
X_train = df_train['sentence']
y_train = df_train['label']

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/ML_DL_datasets/VSFC/human_annotator/test.csv')
df_test.head()

,sentence,label,idx
0,nói tiếng anh lưu loát .,2,0
1,giáo viên rất vui tính .,2,1
2,cô max có tâm .,2,2
3,"giảng bài thu hút , dí dỏm .",2,3
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0,4


In [ ]:
# # Encode labels (positive=0, negative=1, neutral=2)
# le = LabelEncoder()
# df_test['label_encoded'] = le.fit_transform(df_test['label'])
# df_test.head()

In [ ]:
# # Split data
# X_test = df_test['text']
# y_test = df_test['label_encoded']
# Split data
X_test = df_test['sentence']
y_test = df_test['label']

In [ ]:
y_test.unique()

array([2, 0, 1])

In [ ]:
# Split
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

# Chuyển thành Dataset
train_dataset = Dataset.from_pandas(df_train[['sentence', 'label']].rename(columns={'sentence': 'text', 'label_encoded': 'label'}))
val_dataset = Dataset.from_pandas(df_val[['sentence', 'label']].rename(columns={'sentence': 'text', 'label_encoded': 'label'}))
test_dataset = Dataset.from_pandas(df_test[['sentence', 'label']].rename(columns={'sentence': 'text', 'label_encoded': 'label'}))

In [ ]:
# Load tokenizer và model
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenize
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_val = val_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Map:   0%|          | 0/7540 [00:00<?, ? examples/s]

Map:   0%|          | 0/1886 [00:00<?, ? examples/s]

Map:   0%|          | 0/3166 [00:00<?, ? examples/s]

In [ ]:
# Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average="macro")['f1']
    }

In [ ]:
# Training args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Corrected parameter name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    save_strategy="epoch"
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-1649303874.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nguyenan-rio-dev (nguyenan-rio-dev-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.239227,0.931601,0.800223
2,0.323100,0.237730,0.937434,0.821632
3,0.187200,0.268037,0.934252,0.807042


TrainOutput(global_step=1416, training_loss=0.2216039862336412, metrics={'train_runtime': 589.1421, 'train_samples_per_second': 38.395, 'train_steps_per_second': 2.403, 'total_flos': 1482705776104560.0, 'train_loss': 0.2216039862336412, 'epoch': 3.0})

In [ ]:
# Evaluate on test
results = trainer.evaluate(tokenized_test)
print(results)  # Accuracy và F1

{'eval_loss': 0.27393797039985657, 'eval_accuracy': 0.9279848389134555, 'eval_f1_macro': 0.7994947902870724, 'eval_runtime': 14.2627, 'eval_samples_per_second': 221.978, 'eval_steps_per_second': 13.882, 'epoch': 3.0}
